In [1]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import scipy
scipy.__version__

'1.8.1'

In [2]:
## load the kiva_loans.csv. display info and head
df = pd.read_csv('Data/insurance - insurance.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Q1: Do smokers have higher insurance charges than non-smokers?

## 1. State the Hypothesis & Null Hypothesis

 Ho (Null Hypothesis):Everyone has the same insurance charges regardless if they smoke.
 Ha (Alternative Hypothesis): Those that smoke have higher insurance charges than those that don't smoke 

## 2. Determine the correct test to perform.

Type of Data?Numeric
How many groups/samples? 2 groups
Therefore, which test is appropriate? 2 sample T test

# Q2: Are men more likely to smoke than women?

## 1. State the Hypothesis & Null Hypothesis

 Ho (Null Hypothesis): Both men and women are equably as likely to smoke.
 Ha (Alternative Hypothesis): There is a higher likelihood that someone smoke based on their gender. 

## 2. Determine the correct test to perform.

Type of Data? Categorical
How many groups/samples? 2 groups
Therefore, which test is appropriate? Chi-Square

# Q3: Do different regions have different charges, on average?

## 1. State the Hypothesis & Null Hypothesis

 Ho (Null Hypothesis): On average all regions have similar charges.
 Ha (Alternative Hypothesis): There is a big difference in charges based on which region a person is located.

## 2. Determine the correct test to perform.

Type of Data?Numerical
How many groups/samples? 2 groups
Therefore, which test is appropriate? 2 Sample T-Test